# 介绍Polars

## 一、介绍与对比

&emsp;&emsp;Polars的目标是成为一个闪电般快速的数据帧库，利用机器上所有可用的内核。与dask等试图并行化numpy和pandas等现有单线程库的工具不同，polars是从头开始编写的，考虑到数据帧查询的并行化。它通过极大的努力来减少冗余拷贝，高效地遍历内存缓存，并在并行性中最小化争用。

&emsp;&emsp;Polars有lazy和semi-lazy两种API。它可以让你快速地完成大部分工作，就像pandas一样，并且它确实为你提供了一个强大的表达式语法，可以在polars的查询引擎上进行优化和执行。

&emsp;&emsp;在Lazy Polars中，我们可以对整个查询进行查询优化，进一步改善性能和内存压力。

&emsp;&emsp;Polars会按逻辑计划跟踪您的查询。此计划在运行前经过优化和重新排序。当请求一个结果时，Polars会将可用的工作分配给不同的执行者，这些执行者使用eager API中可用的算法来生成结果。因为逻辑计划的优化器和执行者知道整个查询上下文，所以依赖于不同数据源的过程可以动态并行化。

<div align="center">
    <img src="./images/introduction_api.svg" width="30%">
</div>

&emsp;&emsp;Polars的速度非常快，事实上是目前性能最好的解决方案之一。参见`h2oai's db-benchmark`基准测试中的结果。

<div align="center">
    <img src="./images/introduction_db-benchmark.png" width="40%">
</div>

## 二、快速开始
下载`Polars`仅仅是用简单的`pip install`方式。所有二进制文件都是为Python v3.6+预先构建的

In [1]:
import polars as pl

In [2]:
# df = pl.read_csv("https://j.mp/iriscsv")
df = pl.read_csv("./data/iris.csv")
print(df.filter(pl.col("sepal_length") > 5)
      .groupby("species")
      .agg(pl.all().sum()))

shape: (3, 5)
┌─────────────────┬──────────────┬─────────────┬──────────────┬─────────────┐
│ species         ┆ sepal_length ┆ sepal_width ┆ petal_length ┆ petal_width │
│ ---             ┆ ---          ┆ ---         ┆ ---          ┆ ---         │
│ str             ┆ f64          ┆ f64         ┆ f64          ┆ f64         │
╞═════════════════╪══════════════╪═════════════╪══════════════╪═════════════╡
│ Iris-setosa     ┆ 116.9        ┆ 81.7        ┆ 33.2         ┆ 6.1         │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Iris-versicolor ┆ 281.9        ┆ 131.8       ┆ 202.9        ┆ 63.3        │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Iris-virginica  ┆ 324.5        ┆ 146.2       ┆ 273.1        ┆ 99.6        │
└─────────────────┴──────────────┴─────────────┴──────────────┴─────────────┘


正如我们所见，Polars可以漂亮地打印输出对象，包括列名和数据类型作为标题。

### 2.1 支持的数据类型
&emsp;&emsp;polars完全基于Arrow数据类型，并由Arrow内存阵列支持。这使得数据处理缓存高效，并为进程间通信提供了良好的支持。除了Utf8（实际上是大的Utf8）、Category和Object（支持有限），大多数数据类型都遵循Arrow的精确实现。

数据类型包括：
* Int8：8位有符号整数
* Int16: 16位有符号整数
* Int32: 32位有符号整数
* Int64: 64位有符号整数
* UInt8: 8位无符号整数
* UInt16: 16位无符号整数
* UInt32: 32位无符号整数
* UInt64: 64位无符号整数
* Float32：32位浮点数
* Float64：64位浮点数
* Boolean：布尔型有效位压缩
* Utf8：字符串数据（内部实际上是Arrow LargeUtf8）。
* List：列表数组包含包含列表值的子数组和偏移数组。（这实际上是Arrow LargeList内部的做法）。
* Date：日期表示法，内部表示为自UNIX纪元以来的天数，由32位有符号整数编码。
* Datetime：Datetime表示形式，内部表示为自UNIX epoch以来的纳秒，由64位有符号整数编码。
* Duration：timedelate类型。在减去日期/日期时间时创建。
* Time：时间表示法，从午夜开始在内部表示为纳秒。
* Object：受支持的有限数据类型，可以是任何值。

## 三、Lazy Polars & API
如果我们想在Lazy Polars中运行这个查询，我们会写：

In [3]:
print(
    # pl.read_csv("https://j.mp/iriscsv")
    pl.read_csv("./data/iris.csv")
    .lazy()
    .filter(pl.col("sepal_length") > 5)
    .groupby("species")
    .agg(pl.all().sum())
    .collect()
)

shape: (3, 5)
┌─────────────────┬──────────────┬─────────────┬──────────────┬─────────────┐
│ species         ┆ sepal_length ┆ sepal_width ┆ petal_length ┆ petal_width │
│ ---             ┆ ---          ┆ ---         ┆ ---          ┆ ---         │
│ str             ┆ f64          ┆ f64         ┆ f64          ┆ f64         │
╞═════════════════╪══════════════╪═════════════╪══════════════╪═════════════╡
│ Iris-setosa     ┆ 116.9        ┆ 81.7        ┆ 33.2         ┆ 6.1         │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Iris-virginica  ┆ 324.5        ┆ 146.2       ┆ 273.1        ┆ 99.6        │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Iris-versicolor ┆ 281.9        ┆ 131.8       ┆ 202.9        ┆ 63.3        │
└─────────────────┴──────────────┴─────────────┴──────────────┴─────────────┘


当数据未存储在互联网上时，我们也可以使用scan_csv以Lazy Polars运行查询。

In [4]:
print(
    pl.scan_csv("./data/iris.csv")
    .filter(pl.col("sepal_length") > 5)
    .groupby("species")
    .agg(pl.all().sum())
    .collect()
)

shape: (3, 5)
┌─────────────────┬──────────────┬─────────────┬──────────────┬─────────────┐
│ species         ┆ sepal_length ┆ sepal_width ┆ petal_length ┆ petal_width │
│ ---             ┆ ---          ┆ ---         ┆ ---          ┆ ---         │
│ str             ┆ f64          ┆ f64         ┆ f64          ┆ f64         │
╞═════════════════╪══════════════╪═════════════╪══════════════╪═════════════╡
│ Iris-virginica  ┆ 324.5        ┆ 146.2       ┆ 273.1        ┆ 99.6        │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Iris-setosa     ┆ 116.9        ┆ 81.7        ┆ 33.2         ┆ 6.1         │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Iris-versicolor ┆ 281.9        ┆ 131.8       ┆ 202.9        ┆ 63.3        │
└─────────────────┴──────────────┴─────────────┴──────────────┴─────────────┘


&emsp;&emsp;Lazy API构建了一个查询计划。除非明确要求Polars执行查询（通过`LazyFrame.collect()`或`LazyFrame.fetch()`），否则不会执行任何操作。这为Polars提供了查询的整个上下文，允许优化并在给定上下文的情况下选择最快的算法。
从eager变为lazy通常很简单，只需从`.lazy()`开始并以`.collect()`结尾。因此，上面eager的片段将变成：

In [5]:
print(
    df.lazy()
    .filter(pl.col("sepal_length") > 5)
    .groupby("species")
    .agg(pl.all().sum())
    .collect()
)

shape: (3, 5)
┌─────────────────┬──────────────┬─────────────┬──────────────┬─────────────┐
│ species         ┆ sepal_length ┆ sepal_width ┆ petal_length ┆ petal_width │
│ ---             ┆ ---          ┆ ---         ┆ ---          ┆ ---         │
│ str             ┆ f64          ┆ f64         ┆ f64          ┆ f64         │
╞═════════════════╪══════════════╪═════════════╪══════════════╪═════════════╡
│ Iris-versicolor ┆ 281.9        ┆ 131.8       ┆ 202.9        ┆ 63.3        │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Iris-virginica  ┆ 324.5        ┆ 146.2       ┆ 273.1        ┆ 99.6        │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Iris-setosa     ┆ 116.9        ┆ 81.7        ┆ 33.2         ┆ 6.1         │
└─────────────────┴──────────────┴─────────────┴──────────────┴─────────────┘
